In [1]:
import pandas as pd
import numpy as np


In [2]:
dataset = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx'

In [3]:
df_initial = pd.read_excel(dataset,encoding="ISO-8859-1",converters={'Customer ID':str,'Invoice':str})
df_initial.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom


<class 'pandas.core.frame.DataFrame'>
Int64Index: 515255 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      515255 non-null  object        
 1   StockCode    515255 non-null  object        
 2   Description  512327 non-null  object        
 3   Quantity     515255 non-null  int64         
 4   InvoiceDate  515255 non-null  datetime64[ns]
 5   Price        515255 non-null  float64       
 6   Customer ID  407695 non-null  object        
 7   Country      515255 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 35.4+ MB


In [5]:
df_initial.describe()

,Quantity,Price
count,525461.000000,525461.000000
mean,10.337667,4.688834
std,107.424110,146.126914
min,-9600.000000,-53594.360000
25%,1.000000,1.250000
50%,3.000000,2.100000
75%,10.000000,4.210000
max,19152.000000,25111.090000


For this purpose of the exercise, let's first check to see if invoices beginning with 'c' (i.e. cancelled purchases) are in the dataset. Giftware shop is interested in determining customer segmentation based on customers who purchased products, not cancelled purchases, so we can remove all cancelled purchases from the dataset.

In [8]:
df_initial = df_initial[~df_initial.Invoice.str.startswith('C', na=False)]

In [9]:
df_initial.isnull().sum()
#Description & Customer ID contain null records. 

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107560
Country             0
dtype: int64

In [11]:
missing_customerIDs = df_initial['Customer ID'].isnull().sum()/df_initial['Customer ID'].count()

print('Percentage of Customer IDs missing is {:0%}'.format(missing_customerIDs))

Percentage of Customer IDs missing is 26.382467%


In [12]:
#Since ~25% of the records are missing, leaving us with a good portion of data, let's remove these records for now
#Now we are left with fully populated data. 
df_initial.dropna(inplace=True)
df_initial.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [13]:
#Add a new Total Invoice Amt Column
df_initial['Total Line Amount']= df_initial['Quantity'] * df_initial['Price']

In [14]:
df_initial.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Total Line Amount
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0


In [15]:
df_initial['Country'].value_counts()
#Biggest customer is from the UK.

United Kingdom          370951
EIRE                      8507
Germany                   7661
France                    5470
Netherlands               2730
Spain                     1235
Switzerland               1170
Belgium                   1038
Portugal                   984
Sweden                     868
Channel Islands            821
Italy                      710
Australia                  630
Cyprus                     541
Austria                    524
Greece                     512
Denmark                    418
Norway                     365
Finland                    347
United Arab Emirates       315
Unspecified                277
USA                        230
Poland                     182
Malta                      170
Japan                      164
Lithuania                  154
Singapore                  117
Canada                      77
Thailand                    76
Israel                      74
Iceland                     71
RSA                         65
Brazil  

In [16]:
print('Total no. of customers: ',df_initial['Customer ID'].nunique())
print('Total transactions : ',df_initial['Invoice'].nunique())
print('Products sold are : ',df_initial['StockCode'].nunique())

Total no. of customers:  4314
Total transactions done:  19215
Products sold are :  4017


Since we are looking at the data on a customer level, let's aggregate some measures:

1. Total Purchase Amount by Customer 
1. Number of Transactons by Customer
1. Total Number of Distinct Products Purchased by Customer
1. Average Price of All Products Sold to Customer
